In [1]:
# Read necessary output files, compute delays per (car/ride/freight) agent and trip and output the results

import sys
import matsim
import pandas as pd
path_output_basecase = "C:/Users/jdgoe/Documents/Uni/MATSimAdv Wise2324/matsim-berlin-decongestion/output/berlin-v6.0-1pct/"

link_counts = pd.read_csv(path_output_basecase + "analysis/traffic/count_comparison_daily.csv")
link_delays = pd.read_csv(path_output_basecase + "analysis/traffic/traffic_stats_by_link_daily.csv")
network = matsim.read_network(path_output_basecase + "berlin-v6.0." + "output_network.xml.gz")

In [24]:
def get_link_total_delays(
    link_delays: pd.DataFrame, network: pd.DataFrame
) -> pd.DataFrame:
    return(
        link_delays
        .merge(network, how="left", on="link_id")
        .assign(
            total_delays=lambda row: (
                (
                    row["lane_km"] * 1000 / row["freespeed"]
                    - row["lane_km"] * 1000 / row["avg_speed"]
                )
                * row["simulated_traffic_volume"]
            )
        )[
            [
                "link_id",
                "avg_speed",
                "congestion_index",
                "speed_performance_index",
                "simulated_traffic_volume",
                "vol_freight",
                "vol_car",
            ]
        ]
    )

In [7]:
link_delays["link_id"] = link_delays["link_id"].apply(str)

In [13]:
link_counts

link_id                       int64
name                         object
road_type                    object
observed_traffic_volume     float64
simulated_traffic_volume    float64
rel_error                   float64
quality                      object
dtype: object

In [6]:
network_links: pd.DataFrame = network.links.copy()
network_links["link_id"] = network_links["link_id"].apply(str)
network_links

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node
0,31.467991,4.166667,1200.0,1.0,1,"freight,ride,car",1,4864982246,262215600
1,97.996533,6.944444,3200.0,2.0,1,"freight,ride,car",10,26703746,2951553394
2,71.196902,2.777778,600.0,1.0,1,"freight,ride,car",100,27318315,587724650
3,17.031152,4.166667,1200.0,1.0,1,"freight,ride,car",1000,29690627,29690626
4,27.555616,6.944444,1800.0,1.5,1,"freight,ride,car",10000,1106607970,36265803
...,...,...,...,...,...,...,...,...,...
202934,354.868966,12.236861,100000.0,1.0,1,pt,pt_9995,pt_500000065901,pt_500000087401
202935,1353.018128,9.080659,100000.0,1.0,1,pt,pt_9996,pt_500000087401,pt_500000086501
202936,1689.217448,8.082380,100000.0,1.0,1,pt,pt_9997,pt_500000086501,pt_500000086601
202937,1600.265408,7.656772,100000.0,1.0,1,pt,pt_9998,pt_500000086601,pt_500000083701


In [25]:
res = get_link_total_delays(link_delays, network_links)
res

,link_id,avg_speed,congestion_index,speed_performance_index,simulated_traffic_volume,vol_freight,vol_car
0,1,15.000631,1.000000,1.000000,500.0,0.0,500.0
1,10,23.417779,0.907167,0.936699,11900.0,3000.0,8900.0
2,100,9.998863,0.999852,0.999852,100.0,0.0,100.0
3,1000,14.944679,0.996198,0.996198,200.0,0.0,200.0
4,100000,68.518771,0.978782,0.978782,1800.0,100.0,1700.0
...,...,...,...,...,...,...,...
107963,99993,9.876072,0.969776,0.987593,800.0,200.0,600.0
107964,99994,9.997883,0.999773,0.999773,400.0,0.0,400.0
107965,99995,24.724763,0.988867,0.988867,1100.0,0.0,1100.0
107966,99996,24.641157,0.985527,0.985527,1300.0,0.0,1300.0
